#Predicting Olympic medals with Deep Learning

- It's almost Olympics time, and we're here to predict medals!
- Let's start out by installing and importing everything we need:

In [ ]:
!pip install optuna  > /dev/null 2>&1

In [ ]:
!pip install pandas numpy scikit-learn torch transformers  > /dev/null 2>&1

In [ ]:
!pip install wikipedia-api > /dev/null 2>&1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error

import torch
import torch.nn as nn
from transformers import pipeline
from torch.utils.data import DataLoader, TensorDataset

import datetime
import wikipediaapi

If you have a GPU, this is your best friend:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Let's hard-code the participating countries. Each country has a 3-letter code called its NOC. It will be super handy today.

In [ ]:
# List of tuples mapping countries to their NOCs
participating_countries = [
    # Africa
    ("Algeria", "ALG"),
    ("Angola", "ANG"),
    ("Benin", "BEN"),
    ("Botswana", "BOT"),
    ("Burkina Faso", "BUR"),
    ("Burundi", "BDI"),
    ("Cameroon", "CMR"),
    ("Cabo Verde", "CPV"),
    ("Central African Republic", "CAF"),
    ("Chad", "CHA"),
    ("Comoros", "COM"),
    ("Congo", "CGO"),
    ("Democratic Republic of the Congo", "COD"),
    ("Côte d’Ivoire", "CIV"),
    ("Djibouti", "DJI"),
    ("Egypt", "EGY"),
    ("Eritrea", "ERI"),
    ("Eswatini", "SWZ"),
    ("Ethiopia", "ETH"),
    ("Gabon", "GAB"),
    ("Gambia", "GAM"),
    ("Ghana", "GHA"),
    ("Guinea", "GUI"),
    ("Guinea-Bissau", "GBS"),
    ("Equatorial Guinea", "GEQ"),
    ("Kenya", "KEN"),
    ("Lesotho", "LES"),
    ("Liberia", "LBR"),
    ("Libya", "LBA"),
    ("Madagascar", "MAD"),
    ("Malawi", "MAW"),
    ("Mali", "MLI"),
    ("Morocco", "MAR"),
    ("Mauritius", "MRI"),
    ("Mauritania", "MTN"),
    ("Mozambique", "MOZ"),
    ("Namibia", "NAM"),
    ("Niger", "NIG"),
    ("Nigeria", "NGR"),
    ("Uganda", "UGA"),
    ("Rwanda", "RWA"),
    ("Sao Tome and Principe", "STP"),
    ("Senegal", "SEN"),
    ("Seychelles", "SEY"),
    ("Sierra Leone", "SLE"),
    ("Somalia", "SOM"),
    ("South Africa", "RSA"),
    ("South Sudan", "SSD"),
    ("Sudan", "SUD"),
    ("United Republic of Tanzania", "TAN"),
    ("Togo", "TOG"),
    ("Tunisia", "TUN"),
    ("Zambia", "ZAM"),
    ("Zimbabwe", "ZIM"),

    # The Americas
    ("Antigua and Barbuda", "ANT"),
    ("Argentina", "ARG"),
    ("Aruba", "ARU"),
    ("Bahamas", "BAH"),
    ("Barbados", "BAR"),
    ("Belize", "BIZ"),
    ("Bermuda", "BER"),
    ("Bolivia", "BOL"),
    ("Brazil", "BRA"),
    ("Cayman Islands", "CAY"),
    ("Canada", "CAN"),
    ("Chile", "CHI"),
    ("Colombia", "COL"),
    ("Costa Rica", "CRC"),
    ("Cuba", "CUB"),
    ("Dominican Republic", "DOM"),
    ("Dominica", "DMA"),
    ("El Salvador", "ESA"),
    ("Ecuador", "ECU"),
    ("Grenada", "GRN"),
    ("Guatemala", "GUA"),
    ("Guyana", "GUY"),
    ("Haiti", "HAI"),
    ("Honduras", "HON"),
    ("Jamaica", "JAM"),
    ("Mexico", "MEX"),
    ("Nicaragua", "NCA"),
    ("Panama", "PAN"),
    ("Paraguay", "PAR"),
    ("Peru", "PER"),
    ("Puerto Rico", "PUR"),
    ("Saint Kitts and Nevis", "SKN"),
    ("Saint Lucia", "LCA"),
    ("St. Vincent and the Grenadines", "VIN"),
    ("Suriname", "SUR"),
    ("Trinidad and Tobago", "TTO"),
    ("United States", "USA"),
    ("Uruguay", "URU"),
    ("Venezuela", "VEN"),
    ("Virgin Islands, British", "IVB"),
    ("United States Virgin Islands", "ISV"),

    # Asia
    ("Afghanistan", "AFG"),
    ("Bahrain", "BRN"),
    ("Bangladesh", "BAN"),
    ("Bhutan", "BHU"),
    ("Brunei Darussalam", "BRU"),
    ("Cambodia", "CAM"),
    ("China", "CHN"),
    ("Republic of Korea", "KOR"),
    ("Hong Kong, China", "HKG"),
    ("India", "IND"),
    ("Indonesia", "INA"),
    ("Islamic Republic of Iran", "IRI"),
    ("Iraq", "IRQ"),
    ("Japan", "JPN"),
    ("Jordan", "JOR"),
    ("Kazakhstan", "KAZ"),
    ("Kyrgyzstan", "KGZ"),
    ("Kuwait", "KUW"),
    ("Lao People’s Democratic Republic", "LAO"),
    ("Lebanon", "LBN"),
    ("Malaysia", "MAS"),
    ("Maldives", "MDV"),
    ("Mongolia", "MGL"),
    ("Myanmar", "MYA"),
    ("Nepal", "NEP"),
    ("Oman", "OMA"),
    ("Pakistan", "PAK"),
    ("Palestine", "PLE"),
    ("Philippines", "PHI"),
    ("Qatar", "QAT"),
    ("Democratic People’s Republic of Korea", "PRK"),
    ("Saudi Arabia", "KSA"),
    ("Singapore", "SGP"),
    ("Sri Lanka", "SRI"),
    ("Syrian Arab Republic", "SYR"),
    ("Tajikistan", "TJK"),
    ("Chinese Taipei", "TPE"),
    ("Thailand", "THA"),
    ("East Timor", "TLS"),
    ("Turkmenistan", "TKM"),
    ("United Arab Emirates", "UAE"),
    ("Uzbekistan", "UZB"),
    ("Vietnam", "VIE"),
    ("Yemen", "YEM"),

    # Europe
    ("Albania", "ALB"),
    ("Andorra", "AND"),
    ("Armenia", "ARM"),
    ("Austria", "AUT"),
    ("Azerbaijan", "AZE"),
    ("Belgium", "BEL"),
    ("Bosnia and Herzegovina", "BIH"),
    ("Bulgaria", "BUL"),
    ("Cyprus", "CYP"),
    ("Croatia", "CRO"),
    ("Czechia", "CZE"),
    ("Denmark", "DEN"),
    ("Spain", "ESP"),
    ("Estonia", "EST"),
    ("Finland", "FIN"),
    ("France", "FRA"),
    ("Georgia", "GEO"),
    ("Germany", "GER"),
    ("Great Britain", "GBR"),
    ("Greece", "GRE"),
    ("Hungary", "HUN"),
    ("Ireland", "IRL"),
    ("Iceland", "ISL"),
    ("Israel", "ISR"),
    ("Italy", "ITA"),
    ("Kosovo", "KOS"),
    ("Latvia", "LAT"),
    ("Liechtenstein", "LIE"),
    ("Lithuania", "LTU"),
    ("Luxembourg", "LIE"),
    ("North Macedonia", "MKD"),
    ("Malta", "MLT"),
    ("Republic of Moldova", "MDA"),
    ("Monaco", "MON"),
    ("Montenegro", "MNE"),
    ("Netherlands", "NED"),
    ("Norway", "NOR"),
    ("Poland", "POL"),
    ("Portugal", "POR"),
    ("Romania", "ROU"),
    ("San Marino", "SMR"),
    ("Serbia", "SRB"),
    ("Slovakia", "SVK"),
    ("Slovenia", "SLO"),
    ("Sweden", "SWE"),
    ("Switzerland", "SUI"),
    ("Türkiye", "TUR"),
    ("Ukraine", "UKR"),

    # Oceania
    ("American Samoa", "ASA"),
    ("Australia", "AUS"),
    ("Cook Islands", "COK"),
    ("Fiji", "FIJ"),
    ("Guam", "GUM"),
    ("Kiribati", "KIR"),
    ("Marshall Islands", "MHL"),
    ("Federated States of Micronesia", "FSM"),
    ("Nauru", "NRU"),
    ("New Zealand", "NZL"),
    ("Palau", "PLW"),
    ("Papua New Guinea", "PNG"),
    ("Solomon Islands", "SOL"),
    ("Samoa", "SAM"),
    ("Tonga", "TGA"),
    ("Tuvalu", "TUV"),
    ("Vanuatu", "VAN")
]


Sorry *insert name of your elementary school teacher here*, we'll be using Wikipedia!

In [ ]:
#Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(user_agent='My_Olympic_Data_Bot/1.0 (randomemail@gmail.com)')

#Initialize transformer pipeline for text feature extraction
nlp = pipeline("feature-extraction", model="bert-base-uncased", device=device)

Transformers are super cool. Let's see what a transformer can fetch and tell us about different countries from Wikipedia.

In [ ]:
# Function to extract text features from Wikipedia
def get_country_features(country_name):
    page = wiki_wiki.page(country_name)

    if not page.exists():
        print(f"Wikipedia page not found for: {country_name}")
        return None

    # Extract summary text
    summary_text = page.summary

    # Truncate the summary text to fit within the BERT model's limit
    # BERT models typically have a maximum sequence length of 512 tokens
    tokens = summary_text.split()
    tokens = tokens[:512] # Truncate to 512 tokens

    # Use the transformer model to extract features
    features = nlp(summary_text, truncation=True, max_length=512 )

    # Aggregating features (e.g., mean across the tokens)
    aggregated_features = [sum(x)/len(x) for x in zip(*features[0])]

    return aggregated_features

In [ ]:
# Function to compile features for all countries
def compile_country_features():
    all_country_features = {}

    for country_name, NOC in participating_countries:
        features = get_country_features(country_name)
        if features:
            all_country_features[NOC] = features

    return all_country_features

# Compile features for all countries
country_features = compile_country_features()

# Display sample features
print(f"Sample features for United States: {country_features.get('United States')}")

This is why we need NOC's. So we can use our data easily and in a uniform way. Look:

U S A! U S A!

In [ ]:
print(f"Sample features for United States: {country_features.get('USA')}")

You should be getting long vectors of numbers, which are our bert-generated features. If you're getting None here, something is wrong.

In [ ]:
print(f"Sample features for China: {country_features.get('CHN')}")

Ok, now let's look at some data and start preprocessing it. Paris2024 is a Summer Olympics, so we'll only be looking at Summer editions.

In [ ]:
historical_medal_tally_path = '/data/Olympic_Games_Medal_Tally.csv'
paris_2024_medal_tally_path = '/data/Paris_2024_Medal_Table.csv'

# Load the CSV files into dataframes
historical_medal_tally = pd.read_csv(historical_medal_tally_path)
historical_medal_tally.rename(columns={'country_noc': 'country_code'}, inplace=True)

# Filter the data for Summer Olympics only
historical_medal_tally['edition'] = historical_medal_tally['edition'].str.replace(' Olympics', '')
historical_medal_tally[['Year', 'Season']] = historical_medal_tally['edition'].str.split(' ', expand=True)
historical_medal_tally = historical_medal_tally.drop(columns=['year'])
historical_medal_tally_summer = historical_medal_tally[historical_medal_tally['Season'] == 'Summer']
paris_2024_medal_tally = pd.read_csv(paris_2024_medal_tally_path)

participating_countries_df = pd.DataFrame(participating_countries, columns=['country', 'country_code'])


historical_medal_tally_summer


Not all countries have won medals yet, and yet we can't completely disregard them- there might be surprises!

In [ ]:
# Function to add missing countries with 0 medals
def add_missing_countries(medal_tally_df, left_on, right_on):
    # Merge the medal tally with participating countries
    merged_df = pd.merge(participating_countries_df, medal_tally_df, left_on=left_on, right_on=right_on, how='left')

    # Fill NaN values (for countries with no medals) with 0
    merged_df.fillna(0, inplace=True)

    return merged_df

We'll try two test cases: First we'll filter out countries which haven't won medals recently. This is mainly in order to disregard former countries like East Germany, Soviet Russia and such, which were IMMENSE Olympic powerhouses but no longer exist. We don't want them biasing our data...

In [ ]:
# Test case 1: country filtering - predicting only for countries which have won medals in recent Olympics
historical_medal_tally = add_missing_countries(historical_medal_tally_summer, ['country_code'], ['country_code'])
paris_2024_medal_tally = add_missing_countries(paris_2024_medal_tally, ['country_code'], ['country_code'])
historical_medal_tally = historical_medal_tally.drop(columns=['country_y']).rename(columns={'country_x': 'Country'})


historical_medal_tally['Year'] = historical_medal_tally['Year'].astype(int) # Convert 'year' column to integer type
historical_medal_tally

Our second case doesn't do that, but rather looks at all the countries participating in Paris2024.

In [ ]:
## Test case 2: without country filtering - predicting for all participating countries
historical_medal_tally_case_2 = historical_medal_tally
paris_2024_medal_tally_case_2 = paris_2024_medal_tally
historical_medal_tally_case_2

If we think about it, a lot has changed in the world in over a century that the Olympics have been going on. East Germany used to be a powerhouse and no longer exists. The same goes for Yugoslavia and the USSR. Countries have been banned or boycotted, a refugee team and a neutral team were established, and more. In addition, many countries have made great development both in sports and in general.
With that in mind, it might be inaccurate to predict based on over 100 years of data.
Let's filter out data prior to 1992, because things have been more stable since.

We'll have two test cases: One to review only the countries which have won at least 6 medals since 1992, and another to review all participating countries.

In [ ]:

# Test case 1: Keep only countries which have won at least 8 medals since Barcelona 1992 for relevance
historical_medal_tally_filtered = historical_medal_tally[historical_medal_tally['Year'] >= 1992]
medals_by_country = historical_medal_tally_filtered.groupby('country_code')['total'].sum()
countries_with_at_least_6_medals = medals_by_country[medals_by_country >= 6].index
countries_with_at_least_6_medals_df = pd.DataFrame(countries_with_at_least_6_medals)
medals = pd.DataFrame(medals_by_country)
historical_medal_tally_refined = historical_medal_tally_filtered[historical_medal_tally_filtered['country_code'].isin(countries_with_at_least_8_medals)]
paris_2024_medal_tally_refined = paris_2024_medal_tally[paris_2024_medal_tally['country_code'].isin(countries_with_at_least_6_medals)]
historical_medal_tally_df = pd.DataFrame(historical_medal_tally_refined).reset_index(drop=True)
paris_2024_medal_tally_df = pd.DataFrame(paris_2024_medal_tally_refined).reset_index(drop=True)


#Test case 2:
historical_medal_tally_case_2 = historical_medal_tally[historical_medal_tally['Year'] >= 1992]
historical_medal_tally_case_2_df = pd.DataFrame(historical_medal_tally_case_2).reset_index(drop=True)
paris_2024_medal_tally_case_2_refined = paris_2024_medal_tally_case_2
paris_2024_medal_tally_case_2_df = pd.DataFrame(paris_2024_medal_tally_case_2_refined).reset_index(drop=True)


Here are the countries which won medals recently(at least 8 since Sydney2000)

In [ ]:
countries_with_at_least_6_medals_df

Feel free to view the dataframes

In [ ]:
historical_medal_tally_df

In [ ]:
paris_2024_medal_tally_df

We promise, this is as much data processing as we'll be doing today. Pretty straightforward

In [ ]:
# Preprocessing steps to standardize country codes and names for merging

# Standardize column names for easier merging
historical_medal_tally_df.rename(columns={'total': 'Total', 'gold': 'Gold', 'silver': 'Silver', 'bronze': 'Bronze'}, inplace=True)
historical_medal_tally_case_2_df.rename(columns={'total': 'Total', 'gold': 'Gold', 'silver': 'Silver', 'bronze': 'Bronze'}, inplace=True)
paris_2024_medal_tally_df.rename(columns={'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze'}, inplace=True)
paris_2024_medal_tally_case_2_df.rename(columns={'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze'}, inplace=True)
historical_medal_tally_df



Let's only keep the NOC, year and medal tallies. EVerything else isn't so important.

In [ ]:
# Extract relevant columns for merging: ['country_code', 'Total']
historical_medal_tally_relevant = historical_medal_tally_df[['country_code', 'Year', 'Total', 'Gold', 'Silver', 'Bronze']]
historical_medal_tally_relevant_case_2 = historical_medal_tally_case_2_df[['country_code', 'Year', 'Total', 'Gold', 'Silver', 'Bronze']]

In [ ]:
historical_medal_tally_relevant

In [ ]:
historical_medal_tally_relevant_case_2

A quick look at the countries before we do the learning. We can see that the second test case has many more countries because we didn't filter anything out.

In [ ]:
paris_2024_medal_tally_relevant = paris_2024_medal_tally_df[['country_code', 'Total', 'Gold', 'Silver', 'Bronze']]
paris_2024_medal_tally_relevant_case_2 = paris_2024_medal_tally_case_2_df[['country_code', 'Total', 'Gold', 'Silver', 'Bronze']]
paris_2024_unique_codes = paris_2024_medal_tally_relevant['country_code'].unique()
historical_medal_tally_unique_codes = historical_medal_tally_relevant['country_code'].unique()
paris_2024_unique_codes_case_2 = paris_2024_medal_tally_relevant_case_2['country_code'].unique()
historical_medal_tally_unique_codes_case_2 = historical_medal_tally_relevant_case_2['country_code'].unique()


paris_2024_unique_codes, paris_2024_unique_codes_case_2

Remember, this is sequential, so just run everything one by one. The DF views are mainly for debugging. Can you find China?

In [ ]:
# Preparing to merge BERT features with medal tally data
# We already have the country_names from BERT feature extraction which corresponds to the country names
# We will match these with country codes from the medal tally data

# First, let's create a DataFrame for the BERT features with corresponding country codes
bert_feature_df = pd.DataFrame.from_dict(country_features, orient='index')
num_features = len(next(iter(country_features.values())))
bert_feature_df.columns = [f'feature_{i}' for i in range(num_features)]
bert_feature_df['country_code'] = bert_feature_df.index

bert_feature_df.reset_index(drop=True, inplace=True)

# Drop rows with missing country codes (countries that are not in the medal tally data)
bert_feature_df.dropna(subset=['country_code'], inplace=True)

# Now, let's merge the BERT features with both historical and Paris 2024 medal data
# Case 1
historical_data_with_features = pd.merge(historical_medal_tally_relevant, bert_feature_df, on='country_code', how='inner')
paris_2024_data_with_features = pd.merge(paris_2024_medal_tally_relevant, bert_feature_df, on='country_code', how='inner')
# Case 2
historical_data_with_features_case_2 = pd.merge(historical_medal_tally_relevant_case_2, bert_feature_df, on='country_code', how='inner')
paris_2024_data_with_features_case_2 = pd.merge(paris_2024_medal_tally_relevant_case_2, bert_feature_df, on='country_code', how='inner')

historical_data_with_features.shape, paris_2024_data_with_features.shape, historical_data_with_features_case_2.shape, paris_2024_data_with_features_case_2.shape


In [ ]:
historical_data_with_features = pd.DataFrame(historical_data_with_features)
historical_data_with_features

In [ ]:
historical_data_with_features_case_2 = pd.DataFrame(historical_data_with_features_case_2)
historical_data_with_features_case_2

In [ ]:
paris_2024_data_with_features = pd.DataFrame(paris_2024_data_with_features)
paris_2024_data_with_features

In [ ]:
paris_2024_data_with_features_case_2 = pd.DataFrame(paris_2024_data_with_features_case_2)
paris_2024_data_with_features_case_2

#THE MAIN EVENT

Ok, we made it so far.
Let's scale the data and it into train and validation sets. We'll use the ACTUAL Paris results as a test set(should be ready before our presentation). How cool is that?

In [ ]:
scaler1 = StandardScaler()
scaler2 = StandardScaler()
# Splitting features and target (Total medals)
X_train = historical_data_with_features.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
X_train_2 = historical_data_with_features_case_2.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])

X_train = X_train.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
X_train_2 = X_train_2.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

X_train_scaled = scaler1.fit_transform(X_train)
X_train_tensor_scaled = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_train_scaled_2 = scaler2.fit_transform(X_train_2)
X_train_tensor_scaled_2 = torch.tensor(X_train_scaled_2, dtype=torch.float32).to(device)

y_train = historical_data_with_features[['Total', 'Gold', 'Silver', 'Bronze']]
y_train_2 = historical_data_with_features_case_2[['Total', 'Gold', 'Silver', 'Bronze']]

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_train_tensor_2 = torch.tensor(y_train_2.values, dtype=torch.float32).to(device)

paris_2024_data_with_features_sorted = paris_2024_data_with_features.sort_values(by='Gold', ascending=False)
paris_2024_data_with_features_sorted_2 = paris_2024_data_with_features_case_2.sort_values(by='Gold', ascending=False)

X_test = paris_2024_data_with_features_sorted.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
X_test_2 = paris_2024_data_with_features_sorted_2.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
test_countries = paris_2024_data_with_features_sorted['country_code']
test_countries_2 = paris_2024_data_with_features_sorted_2['country_code']

# Add a constant 'Year' column to X_test with value 2024
X_test.insert(0, 'Year', 2024)
X_test_2.insert(0, 'Year', 2024)
#Scale and transform
X_test_scaled = scaler1.transform(X_test)  # Use the same scaler fitted on training data
X_test_tensor_scaled = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_test_scaled_2 = scaler2.transform(X_test_2)  # Use the same scaler fitted on training data
X_test_tensor_scaled_2 = torch.tensor(X_test_scaled_2, dtype=torch.float32).to(device)

y_test = paris_2024_data_with_features_sorted[['Total', 'Gold', 'Silver', 'Bronze']]
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

y_test_2 = paris_2024_data_with_features_sorted_2[['Total', 'Gold', 'Silver', 'Bronze']]
y_test_tensor_2 = torch.tensor(y_test_2.values, dtype=torch.float32).to(device)






#TRAINING

We'll use Optuna in order to tuna Hyperparameters and find the best combo. Then we'll train our network and see the results. It's as simple as that.

In [ ]:
import optuna

# Splitting train and validation data
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train_tensor_scaled, y_train_tensor, test_size=0.2, random_state=42)

def objective(trial):
    # Define the hyperparameter search space
    n_units_1 = trial.suggest_int('n_units_1', 256, 1024)
    n_units_2 = trial.suggest_int('n_units_2', 128, 512)
    n_units_3 = trial.suggest_int('n_units_3', 64, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    # Building the model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(X_train_sub.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(X_train_sub).to(device)
        loss = criterion(outputs.squeeze(), y_train_sub)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        loss.backward()
        optimizer.step()

    # Evaluate the model on validation data
    with torch.no_grad():
        val_outputs = model(X_val).to(device)
        val_outputs = torch.round(val_outputs)  # Round the predictions to the nearest integer
        val_mae = {}
        val_accuracy = {}

        for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
          mae = torch.mean(torch.abs(val_outputs[:,i] - y_val[:,i])).item()
          val_mae[medal_type] = mae
          correct = torch.sum(torch.abs(val_outputs[:,i] - y_val[:,i]) <= torch.tensor(tolerance_biases[i]).to(device)*y_val[:,i])
          accuracy = correct.item() / len(y_val) * 100
          val_accuracy[medal_type] = accuracy
          val_loss = criterion(val_outputs[:,i], y_val[:,i])
        print(f"Validation Loss: {val_loss} ; Validation Accuracy: {accuracy}%")

    return sum(val_mae.values())  # Return the validation MAE as a Python numbe

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Build and evaluate the model with the best hyperparameters
n_units_1 = best_params['n_units_1']
n_units_2 = best_params['n_units_2']
n_units_3 = best_params['n_units_3']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
tolerance = 0.2
tolerance_biases = [6.0, 2.0, 2.0, 2.0]

    # Building the model with trial hyperparameters
best_model = nn.Sequential(
        nn.Linear(X_train_tensor_scaled.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

# Train with the entire training data and evaluate on the test set
losses = []
for epoch in range(50):
    optimizer.zero_grad()
    outputs = best_model(X_train_tensor_scaled).to(device)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    losses.append(loss.item())
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    optimizer.step()

    # Evaluate the model on test data
with torch.no_grad():
    test_mae = {}
    test_accuracy = {}
    test_outputs = best_model(X_test_tensor_scaled).to(device)
    test_outputs = torch.round(test_outputs)  # Round the predictions to the nearest integer
    for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
      test_loss = criterion(test_outputs[:,i], y_test_tensor[:,i])
      correct = torch.sum(torch.abs(test_outputs[:,i] - y_test_tensor[:,i]) <= torch.max(tolerance*y_test_tensor[:,i], torch.tensor(tolerance_biases[i]).to(device)))
      accuracy = correct.item() / len(y_test) * 100
      test_accuracy[medal_type] = accuracy
      mae = torch.mean(torch.abs(test_outputs[:,i] - y_test_tensor[:,i])).item()  # Calculate MAE
      test_mae[medal_type] = mae


print(f"Test MAE: {test_mae}")
print(f"Test Loss: {test_loss} ; Test Accuracy: {test_accuracy}%")
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs for Test Case 1')
plt.show()


Let's see what the results are. We already have predictions for each medal separately and total medals in the previous cell. Pretty accurate, right? Wait till you see when happens in the second test case.

In [ ]:
medal_manual_sum = torch.sum(test_outputs[:,1:], dim=1)
sum_correct = torch.sum(torch.abs(medal_manual_sum - y_test_tensor[:,0]) <= torch.max(tolerance*y_test_tensor[:,0], torch.tensor(6.0).to(device)))
sum_accuracy = sum_correct.item() / len(y_test) * 100

print(f"Test Accuracy for Manual Medal Total: {sum_accuracy}%")

In [ ]:
# The Olympic medal table is ranked by and order of precedence: Gold, then Silver, then Bronze. We'll apply this precedence to our accuracy as well.
medal_importance_weight = [0.4, 0.3, 0.2, 0.1]
medal_table_accuracy_case_1 = test_accuracy['Gold']*medal_importance_weight[0] + test_accuracy['Silver']*medal_importance_weight[1] + test_accuracy['Bronze']*medal_importance_weight[2] + test_accuracy['Total']*medal_importance_weight[3]
print(f"Test Accuracy for Medal Table with Medal Precedence: {medal_table_accuracy_case_1}%")

No let's see the predicted table!

In [ ]:
test_countries = np.array(test_countries)  # Convert to a numpy array if it's not already

# Create a DataFrame from the test predictions
predictions_df = pd.DataFrame({
    'Country': test_countries,
    'Predicted Gold': test_outputs[:, 1].cpu().numpy(),
    'Predicted Silver': test_outputs[:, 2].cpu().numpy(),
    'Predicted Bronze': test_outputs[:, 3].cpu().numpy(),
    'Predicted Total': test_outputs[:, 0].cpu().numpy(),
    'Total Predicted': medal_manual_sum.cpu().numpy()
})

predictions_df = predictions_df.sort_values(by=['Predicted Gold', 'Predicted Silver', 'Predicted Bronze'], ascending=False).reset_index(drop=True)
predictions_df.index += 1
predictions_df


And for reference, the actual table(added at the end of the Olympics):

In [ ]:
actual_df = pd.DataFrame({
    'Country': test_countries,
    'Actual Gold': y_test['Gold'].values,
    'Actual Silver': y_test['Silver'].values,
    'Actual Bronze': y_test['Bronze'].values,
    'Actual Total': y_test['Total'].values
})
actual_df = actual_df.sort_values(by=['Actual Gold', 'Actual Silver', 'Actual Bronze'], ascending=False).reset_index(drop=True)
actual_df.index += 1
actual_df

Ok, now for the second test case. Here we didn't filter anything out, just looked at all the participating countries in Paris.

In [ ]:
# Test case 2:

# Splitting train and validation data
X_train_sub_2, X_val_2, y_train_sub_2, y_val_2 = train_test_split(X_train_tensor_scaled_2, y_train_tensor_2, test_size=0.2, random_state=42)

def objective(trial):
    # Define the hyperparameter search space
    n_units_1 = trial.suggest_int('n_units_1', 256, 1024)
    n_units_2 = trial.suggest_int('n_units_2', 128, 512)
    n_units_3 = trial.suggest_int('n_units_3', 64, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    # Building the model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(X_train_sub.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(X_train_sub_2).to(device)
        loss = criterion(outputs.squeeze(), y_train_sub_2)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        loss.backward()
        optimizer.step()

    # Evaluate the model on validation data
    with torch.no_grad():
        val_outputs = model(X_val_2).to(device)
        val_outputs = torch.round(val_outputs)  # Round the predictions to the nearest integer
        val_mae = {}
        val_accuracy = {}
        tolerance = 0.2
        tolerance_biases = [6.0, 2.0, 2.0, 2.0]
        for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
          mae = torch.mean(torch.abs(val_outputs[:,i] - y_val_2[:,i])).item()
          val_mae[medal_type] = mae
          correct = torch.sum(torch.abs(val_outputs[:,i] - y_val_2[:,i]) <= torch.tensor(tolerance_biases[i]).to(device)*y_val_2[:,i])
          accuracy = correct.item() / len(y_val_2) * 100
          val_accuracy[medal_type] = accuracy
          val_loss = criterion(val_outputs[:,i], y_val_2[:,i])
        print(f"Validation Loss: {val_loss} ; Validation Accuracy: {accuracy}%")

    return sum(val_mae.values())  # Return the validation MAE as a Python numbe

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Build and evaluate the model with the best hyperparameters
n_units_1 = best_params['n_units_1']
n_units_2 = best_params['n_units_2']
n_units_3 = best_params['n_units_3']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
tolerance = 0.2
tolerance_biases = [6.0, 2.0, 2.0, 2.0]

    # Building the model with trial hyperparameters
best_model = nn.Sequential(
        nn.Linear(X_train_tensor_scaled_2.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

# Train with the entire training data and evaluate on the test set
losses = []
for epoch in range(50):
    optimizer.zero_grad()
    outputs = best_model(X_train_tensor_scaled_2).to(device)
    loss = criterion(outputs.squeeze(), y_train_tensor_2)
    loss.backward()
    losses.append(loss.item())
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    optimizer.step()

    # Evaluate the model on test data
with torch.no_grad():
    test_mae = {}
    test_accuracy_2 = {}
    test_outputs_2 = best_model(X_test_tensor_scaled_2).to(device)
    test_outputs_2 = torch.round(test_outputs_2)  # Round the predictions to the nearest integer
    for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
      test_loss = criterion(test_outputs_2[:,i], y_test_tensor_2[:,i])
      correct = torch.sum(torch.abs(test_outputs_2[:,i] - y_test_tensor_2[:,i]) <= torch.max(tolerance*y_test_tensor_2[:,i], torch.tensor(tolerance_biases[i]).to(device)))
      accuracy = correct.item() / len(y_test_2) * 100
      test_accuracy_2[medal_type] = accuracy
      mae = torch.mean(torch.abs(test_outputs_2[:,i] - y_test_tensor_2[:,i])).item()  # Calculate MAE
      test_mae[medal_type] = mae


print(f"Test MAE: {test_mae}")
print(f"Test Loss: {test_loss} ; Test Accuracy: {test_accuracy_2}%")
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs for Test Case 2')
plt.show()


Wow, that's crazy! Let's sum them up:

In [ ]:
medal_manual_sum_2 = torch.sum(test_outputs_2[:,1:], dim=1)
sum_correct = torch.sum(torch.abs(medal_manual_sum_2 - y_test_tensor_2[:,0]) <= torch.max(tolerance*y_test_tensor_2[:,0], torch.tensor(6.0).to(device)))
sum_accuracy_2 = sum_correct.item() / len(y_test_2) * 100

print(f"Test Accuracy for Manual Medal Total in Test Case 2: {sum_accuracy_2}%")

In [ ]:
# The Olympic medal table is ranked by and order of precedence: Gold, then Silver, then Bronze. We'll apply this precedence to our accuracy as well.
medal_importance_weight = [0.4, 0.3, 0.2, 0.1]
medal_table_accuracy_case_2 = test_accuracy_2['Gold']*medal_importance_weight[0] + test_accuracy_2['Silver']*medal_importance_weight[1] + test_accuracy_2['Bronze']*medal_importance_weight[2] + test_accuracy_2['Total']*medal_importance_weight[3]
print(f"Test Accuracy for Medal Table with Medal Precedence: {medal_table_accuracy_case_2}%")

And for the medal table...

In [ ]:
test_countries_2 = np.array(test_countries_2) #Convert to np array to put in a dataframe

# Create a DataFrame from the test predictions
predictions_case_2_df = pd.DataFrame({
    'Country': test_countries_2,
    'Predicted Gold': test_outputs_2[:, 1].cpu().numpy(), #Move to cpu to use numpy
    'Predicted Silver': test_outputs_2[:, 2].cpu().numpy(),
    'Predicted Bronze': test_outputs_2[:, 3].cpu().numpy(),
    'Predicted Total': test_outputs_2[:, 0].cpu().numpy(),
    'Total Predicted': medal_manual_sum_2.cpu().numpy()
})

predictions_case_2_df = predictions_case_2_df.sort_values(by=['Predicted Gold', 'Predicted Silver', 'Predicted Bronze'], ascending=False).reset_index(drop=True)
predictions_case_2_df.index += 1
predictions_case_2_df

Compare to the actual results:

In [ ]:
actual_df_2 = pd.DataFrame({
    'Country': test_countries_2,
    'Actual Gold': y_test_2['Gold'].values,
    'Actual Silver': y_test_2['Silver'].values,
    'Actual Bronze': y_test_2['Bronze'].values,
    'Actual Total': y_test_2['Total'].values,
})

actual_df_2 = actual_df_2.sort_values(by=['Actual Gold', 'Actual Silver', 'Actual Bronze'], ascending=False).reset_index(drop=True)
actual_df_2.index += 1
actual_df_2

And that's it for today. See you in 2028(or 2026 but then you'll have to go up and swap "summer" out for "winter")!